In [5]:
import numpy as np
from pygsm import GSMObserver
from pygsm import GlobalSkyModel
import healpy
import matplotlib.pyplot as plt
from scipy import interpolate

#Code below is just data extraction 
tag='100mhz'
fname=tag+'/results_pattern_' + tag + '_total90.dat'
dat=np.loadtxt(fname,delimiter=',')
f=open(fname,'r')
ll=f.readline()[:-1]
f.close()

ii=ll.find(',')
ll=ll[ii+2:]
ii=ll.find(',')
ll=ll[ii+2:]
freqs=np.fromstring(ll,sep=',')[:-1]/1e6


th=dat[:,0]
phi=dat[:,1]

nth=len(np.unique(th))
nphi=len(np.unique(phi))

th=th*np.pi/180
phi=phi*np.pi/180

thmat=np.reshape(th,[nphi,nth])
phimat=np.reshape(phi,[nphi,nth])
th=thmat[0,:]
phi=phimat[:,0]

dat=dat[:,2:]
nspec=dat.shape[1]

In [6]:
def almsdotalms(alms1,alms2,nside=None):
    """
    Function that convolutes two alm spaces together 
    """
    nalm=len(alms2)
    lmax=np.int(np.sqrt(2*nalm))
    if nside is None:
        nside=np.int(lmax/3)
    if len(alms1.shape)==1:
        ans=2*np.dot(np.conj(alms1[lmax:]),alms2[lmax:])
        ans=ans+np.dot(alms1[:lmax],alms2[:lmax])
    else:
        ans=2*np.dot(np.conj(alms1[lmax:,:].T),alms2[lmax:])
        ans=ans+np.dot(alms1[:lmax,:].T,alms2[:lmax])
    return np.real(ans)*healpy.nside2npix(nside)/4/np.pi

def rotate_map(map,lat):
    myrot=healpy.rotator.Rotator([0,0,90-lat]);
    return myrot.rotate_map_pixel(map)

In [7]:
#Picks an arbitrary nside (needs to be a power of 2)
nside=256
#Give the number of pixels for the given nside.
npix=healpy.nside2npix(nside)
#Gives the angle corresponding to each pixel (in rad)
hp_th,hp_phi=healpy.pix2ang(nside,np.arange(npix))

#Latitude here
lat=-47


#Find the indices at which theta changes
inds=np.where(np.diff(hp_th)!=0)[0]
inds=np.append(0,inds+1)

#
beams=np.zeros([len(hp_th),nspec])

#
tmp=healpy.sphtfunc.map2alm(beams[:,0])
nalm=len(tmp)
lvec=np.zeros(nalm,dtype='int64')
mvec=np.zeros(nalm,dtype='int64')
lmax=np.int(np.sqrt(2*nalm))

icur=0
for m in range(lmax):
    nn=lmax-m
    mvec[icur:icur+nn]=m
    lvec[icur:icur+nn]=np.arange(m,lmax)
    icur=icur+(lmax-m)
    
alms=np.zeros([nalm,nspec],dtype='complex')
beam_norm=np.zeros(nspec)

#loop over frequencies, then loop over theta values to create a set
#of beams interpolated onto healpix coordinates.  Then take their
#spherical harmonic transforms.
for ii in range(nspec):
    mat=10**(np.reshape(dat[:,ii],[nphi,nth])/10)

    myinterp=interpolate.interp2d(th,phi,mat,'cubic',fill_value=0)
    hp_beam=np.zeros(len(hp_th))

    for i in range(np.int(len(inds)/2)+2):
        i1=inds[i]
        i2=inds[i+1]                                                       
        hp_beam[i1:i2]=myinterp(hp_th[i1],hp_phi[i1:i2])[:,0]
    beam_norm[ii]=np.sqrt(np.sum(hp_beam**2))
    #beams[:,ii]=rotate_map(hp_beam/np.sqrt(np.sum(hp_beam**2)),lat)
    beams[:,ii]=rotate_map(hp_beam/beam_norm[ii],lat)
    #beams[:,ii]=hp_beam/np.sqrt(np.sum(hp_beam**2))
    alms[:,ii]=healpy.sphtfunc.map2alm(beams[:,ii])
    #assert(1==0)

numin=55
numax=105
alms_scaled=alms.copy()

imin=np.min(np.where(freqs>numin))
imax=np.max(np.where(freqs<numax))

mycov_cl=np.real(2*np.dot(np.conj(alms_scaled[lmax:,:].T),alms_scaled[lmax:,:])+np.dot(alms_scaled[:lmax,:].T,alms_scaled[:lmax,:]))
ee,vv=np.linalg.eig(mycov_cl[imin:imax,imin:imax])

mvec=np.zeros(nalm,dtype='int64')

beam_modes=np.dot(alms[:,imin:imax],vv)

#Transforms Gallactic coordinates to Elliptical coordinates 
myrot=healpy.rotator.Rotator(coord=['G','E'])

#This part of the code generates the GSM map using pygsm 
gsm = GlobalSkyModel()

#At frequency of 80MHz
gsm_map_org=gsm.generate(80)
#Degrades the map to the predefined nside of the map
gsm_map_org=healpy.ud_grade(gsm_map_org,nside)
#Rotate a HEALPix map to a new reference frame in pixel space
gsm_map=myrot.rotate_map_pixel(gsm_map_org)

#Converts the GSM sky model map to alms coefficients 
alms_gsm=healpy.map2alm(gsm_map)

#Hour angle 
nha=96
ha=np.arange(nha)/(1.0*nha)*2*np.pi

#Resultant amplitude when multiplying both alms' together 
amps=np.zeros([beam_modes.shape[1],nha])

#To rotate the map, we also multiply by a factor of e^(i(ha)phi)
for i in range(nha):
    amps[:,i]=almsdotalms(beam_modes,alms_gsm*np.exp(1J*ha[i]*mvec),nside=nside)

In [84]:
amps.shape
amps
alms_gsm

array([ 7.05158192e+03+0.j        ,  4.66883484e+02+0.j        ,
       -6.37567706e+02+0.j        , ..., -1.74368785e-02+0.00104719j,
        7.83712492e-04-0.00275715j,  4.66199574e-03-0.00651539j])

In [77]:
maps_nha = np.zeros([beam_modes.shape[1],nha])
#beam_mode = 1
for i in range(nha):
    maps_nha[i] = healpy.alm2map(amps[:,i],nside=nside)

Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin


ValueError: Array must be C contiguous for this operation.